<a href="https://colab.research.google.com/github/usesnames/Reviews_analyzer/blob/main/sentiment_intesa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [242]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import os
import pandas as pd
import numpy as np
import json
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

import tensorflow as tf

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Acquisizione dati

In [ ]:
!pip install -qq google-play-scraper
#!pip install app-store-scraper
from google_play_scraper import Sort, reviews, app, reviews_all
app_name = 'com.latuabancaperandroid'

Helper function that prints JSON objects a bit better:

In [ ]:
def print_json(json_object):
  json_str = json.dumps(
    json_object, 
    indent=2, 
    sort_keys=True, 
    default=str
  )
  print(json_str)

In [ ]:
app_reviews = []

for score in list(range(1, 6)):
  rvs, _ = reviews(
    app_name,
    lang='it',
    country='it',
    sort=Sort.NEWEST,
    count= 4000, 
    filter_score_with=score
  )
  for r in rvs:
    r['appId'] = app_name
  app_reviews.extend(rvs)

In [ ]:
print_json(app_reviews[0])

In [ ]:
len(app_reviews)

# Parametri

# Preprocessing

In [143]:
if not os.path.isfile('balanced_reviews.csv'):
  balanced_reviews_df = pd.DataFrame(app_reviews)
  balanced_reviews_df.to_csv('balanced_reviews.csv', index=None, header=True, sep='\t')
  print('generato file balanced_reviews.csv')

balanced_reviews_df = pd.read_csv('balanced_reviews.csv', sep='\t')
balanced_reviews_df.drop(labels=['userName', 'userImage', 'replyContent', 'repliedAt', 
                                 'at', 'appId', 'reviewCreatedVersion', 'reviewId'], 
                         axis=1, inplace=True)

In [154]:
def remove_stop_words(dataset):
    stop_words = set(stopwords.words('italian'))
    for i in dataset:
        i = ([token.lower() for token in i if token not in stop_words])
    return dataset

def normalize(dataset):
    lemmatizer = nltk.stem.snowball.ItalianStemmer(ignore_stopwords=False)
    for i in dataset:
        i = " ".join([lemmatizer.stem(token) for token in i]).strip()
    return dataset

def remove_garbage(dataset):
    garbage = "~`!@#$%^&*()_-+={[}]|\:;'<,>.?/"
    for i in dataset:
      i = "".join([char for char in i if char not in garbage])
    return dataset

def fit_corpus(train_data):
    tfidf = TfidfVectorizer(min_df=10, max_df=0.6, ngram_range=(1,2))
    tfidf.fit(train_data)
    return tfidf

def transform_data(tfidf, dataset):
    features = tfidf.transform(dataset)
    return pd.DataFrame(features.todense(), columns = tfidf.get_feature_names())

In [155]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(balanced_reviews_df['content'], balanced_reviews_df['score'], 
                                                    test_size=0.3, random_state=42, stratify=balanced_reviews_df['score'])
y_train = y_train.map(lambda num_stars: 1 if (num_stars == 5 or num_stars == 4) else 0)
y_test = y_test.map(lambda num_stars: 1 if (num_stars == 5 or num_stars == 4) else 0)

def preprocess(data, tfidf=None):
  data.map(lambda el: nltk.tokenize.word_tokenize(el))
  data = remove_stop_words(data)
  data = normalize(data)
  data = remove_garbage(data)
  if tfidf==None:
    tfidf = fit_corpus(data) #Fitting the vecorizer
  features = transform_data(tfidf, data)
  return features, tfidf

In [156]:
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
train_features, tfidf = preprocess(X_train)  #transforming 
test_features, _ = preprocess(X_test, tfidf)    

#Training

In [174]:
clf = LogisticRegression(random_state=0, solver='lbfgs', C=0.6, tol=0.0001, max_iter=1000)
clf.fit(train_features, y_train)

LogisticRegression(C=0.6, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [175]:
clf.score(test_features, y_test)

0.8841666666666667

In [246]:
demo_review = "bella e utile, ma troppi crash"

prediction = clf.predict(preprocess(pd.Series([demo_review]), tfidf)[0])[0]
if prediction==1:
  print("recensione postiva")
else:
  print("recensione non positiva")

recensione non positiva


#Serialization

In [223]:
# create a TF model with the same architecture
tf_model = tf.keras.models.Sequential()
tf_model.add(tf.keras.Input(shape=(len(tfidf.vocabulary_),)))
tf_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# assign the parameters from sklearn to the TF model
tf_model.layers[0].weights[0].assign(clf.coef_.transpose())
tf_model.layers[0].bias.assign(clf.intercept_)

<tf.Variable 'UnreadVariable' shape=(1,) dtype=float32, numpy=array([-0.22950141], dtype=float32)>

In [247]:
predictions = clf.predict(preprocess(X_test, tfidf)[0])
tf_predictions = tf_model(np.array(preprocess(X_test, tfidf)[0])) 

flag="stesse predictions"
for el, tf_el in zip(predictions, tf_predictions.numpy()):
  if((el==1 and tf_el[0]<0.5) or (el==0 and tf_el[0]>0.5)):
    flag="predicitons divergono"
print(flag)

stesse predictions


In [241]:
tf_model.save('saved_model')

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model') 
tflite_model = converter.convert()

# Save the model.
with open('sentiment_model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: saved_model/assets
